In [80]:
%matplotlib inline

# imports
import pandas as pd
import numpy as np

from PIL import Image
import os

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [64]:
# const

dir_path = "../dataset/brain-tumor-mri"
#dir_path = "/kaggle/input/brain-tumor-mri-dataset"

labels = ["notumor", "glioma", "meningioma", "pituitary"]

train_percent = .7
val_percent = .2
test_percent = .1

image_size = 128

random_state = 1

batch_size = 64
learning_rate = 0.001
num_epoch = 200

# Data Manipulation

In [65]:
# creamos un dataset de la forma (path, class)
dataset = pd.DataFrame(columns = ["path", "label"])


for dir in os.listdir(dir_path):
    for label in os.listdir(os.path.join(dir_path, dir)):
        for img in os.listdir(os.path.join(dir_path, dir, label)):
            dataset.loc[len(dataset)] = {"path": os.path.join(dir_path, dir, label, img),
                                   "label": labels.index(label) }


print("num elementos:", dataset.count(axis=1).size)
print(dataset.head())

num elementos: 7023
                                                path cat
0  ../dataset/brain-tumor-mri/Training/pituitary/...   3
1  ../dataset/brain-tumor-mri/Training/pituitary/...   3
2  ../dataset/brain-tumor-mri/Training/pituitary/...   3
3  ../dataset/brain-tumor-mri/Training/pituitary/...   3
4  ../dataset/brain-tumor-mri/Training/pituitary/...   3


In [66]:
min_cat_size = dataset.label.value_counts().min()

print("> Cantidad de imagenes de la categoria que menos tiene:", min_cat_size)

> Cantidad de imagenes de la categoria que menos tiene: 1621


In [67]:
# Hacemos que todos tengan los mismos elementos
dataset = pd.DataFrame(dataset.groupby("label").apply(lambda label: label.sample(min_cat_size, random_state=random_state)).reset_index(drop=True))

print("> Cantidad de elementos: ", dataset.count(axis=1).size)
print(dataset.label.value_counts())
print(dataset.head())

> Cantidad de elementos:  6484
0    1621
1    1621
2    1621
3    1621
Name: cat, dtype: int64
                                                path cat
0  ../dataset/brain-tumor-mri/Training/notumor/Tr...   0
1  ../dataset/brain-tumor-mri/Testing/notumor/Te-...   0
2  ../dataset/brain-tumor-mri/Training/notumor/Tr...   0
3  ../dataset/brain-tumor-mri/Training/notumor/Tr...   0
4  ../dataset/brain-tumor-mri/Training/notumor/Tr...   0


In [75]:
# separamos en los cjtos de entrenamiento
train_dataset, tmp = train_test_split(dataset, train_size=train_percent, stratify=dataset["label"], shuffle=True, random_state=random_state)
val_dataset, test_dataset = train_test_split(tmp, test_size=test_percent/(test_percent+val_percent), stratify=tmp["label"], shuffle=True, random_state=random_state)

print("> train", train_dataset.count(axis=1).size, train_dataset.count(axis=1).size/dataset.count(axis=1).size)
print(train_dataset.label.value_counts())
print()

print("> val", val_dataset.count(axis=1).size, val_dataset.count(axis=1).size/dataset.count(axis=1).size)
print(val_dataset.label.value_counts())
print()

print("> test", test_dataset.count(axis=1).size, test_dataset.count(axis=1).size/dataset.count(axis=1).size)
print(test_dataset.label.value_counts())
print()

> train 4538 0.6998766193707588
2    1135
3    1135
0    1134
1    1134
Name: cat, dtype: int64

> val 1297 0.2000308451573103
0    325
2    324
1    324
3    324
Name: cat, dtype: int64

> test 649 0.10009253547193091
1    163
3    162
0    162
2    162
Name: cat, dtype: int64



# Preparations

## Classes

In [79]:
class CustomDataset(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx) -> (str, int):

        row = self.dataframe.iloc[idx]
        img, label = row.path, row.label

        img = Image.open(img)
        if self.transform: img = self.transform(img)

        return img, label

In [ ]:
class EarlyStopping:
    
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta

        self.early_stop = False
        self.counter = 1

        self.best_model = None
        self.best_loss = float("inf")

def __call__(self, val_loss, model):

    if val_loss < (self.best_loss - self.min_delta):
        self.best_loss = val_loss
        self.counter = 0

        self.best_model = model.state_dict()
        return True


    self.counter += 1
    if self.counter >= self.patience: return True

def reset(self):
    self.early_stop = False
    self.counter = 1

In [85]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        
        # Capas convolucionales
        self.convin = nn.Conv2d( 3, 64, kernel_size=3, stride=1, padding=1)
        self.conv1 = nn.Conv2d( 64, 64, kernel_size=3, stride=1, padding=1)
        self.convout = nn.Conv2d( 64, 128, kernel_size=3, stride=1, padding=1)

        # Capas Fully connected 
        # ajustado para 128
        self.fcin = nn.Linear(in_features=128 * 16 * 16, out_features=128)
        self.fcout = nn.Linear(in_features=len(labels), out_features=4)

        # BatchNorm2d
        self.bn1 = nn.BatchNorm2d(64)
        self.bnout = nn.BatchNorm2d(64)
        
        # Max pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Capa Dropout 
        self.dropout25 = nn.Dropout(0.25)
        
            

    def forward(self, x):

        # conv in
        x = self.convin(x)
        in_copy = x.clone()
        x = self.pool(F.relu(x))

        # conv 1
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = x.dropout25(x)

        # conv out
        x = self.pool(F.relu(self.bnout(self.convout(x + in_copy)))) 
        x = self.dropout25(x)
    
        # flatten
        x = x.view(-1, 128*16*16)

        # fully in
        x = F.relu(self.fcin(x))
        x = self.dropout25(x)

        # fully out
        x = F.relu(self.fc2out(x))
        
        return x

## Functions

In [81]:
def eval(model, loader, *, device = None, criterion = None):
    """returns pred, real, accuracy, val_loss"""
    if not (device and criterion): raise Exception("Params needed")

    model.eval()

    corrects = 0

    pred = []
    real = []
    loss = 0

    with torch.no_grad():
        for imgs, labels in loader:

            imgs, labels = imgs.to(device), labels.to(device)

            outs = model(imgs)
            _, predicted = torch.max(outs, 1)

            loss += criterion(outs, labels).item()

            pred.extend(predicted.tolist())
            real.extend(labels.tolist())

            corrects = (predicted == labels).sum().item()
    
    return  pred, real, loss/len(loader), corrects/len(loader)

In [84]:
def train_loop(model, loader, *, optimizer, criterion, num_epoch=100, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), train_loss_h=[], train_acc_h=[], val_loss_h=[] ,val_acc_h=[], early_stopping, callback=lambda **_: None, early_callback=lambda **_: None):
    if not (optimizer and criterion and early_stopping): raise Exception("Params needed")

    message_controler = len(loader)/5

    for epoch in range(num_epoch):

        running_loss = 0.0
        corrects = 0
        total = 0

        for i, (imgs, labels) in enumerate(loader, 1):
            model.train()

            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()

            outs = model(imgs)
            loss = criterion(outs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # extra for analytics
            _, predicted = torch.max(outs, 1)
            total += labels.size()
            corrects += (predicted == labels).sum().item()
            # ==================

            if not i % message_controler:
                print(f"Epoch {epoch+1}, Batch {i}, Loss, {running_loss/message_controler:.4f}")
            
        preds, labels, val_loss, val_acc  = eval(model, val_dataset, device=device, criterion=criterion)

        # extra for analytics
        train_loss = running_loss /len(loader)
        train_loss_h.append(train_loss)
        train_acc = 100*corrects/total
        train_acc_h.append(train_acc)

        val_loss_h.append(val_loss)
        val_acc_h.append(val_acc)
        # ===================

        callback(val_acc=val_acc, val_loss=val_loss, train_loss=train_loss, train_acc=train_acc)

        if(early_stopping(val_loss, model)): early_callback(model=model, early_stopping=early_stopping)

    return train_acc_h, val_acc_h, train_loss_h, val_loss_h, preds, labels

# Utils

In [ ]:
def save_model(model, path, complete=False):
    try:
        torch.save(model if complete else model.state_dict(), path)
    except Exception as e:
        print("Error al guardar el model:", e)

In [ ]:
def show_img(img, label):
    print('Label: ', dataset.cat[label], "(" + str(label) + ")")
    plt.imshow(img.permute(1, 2, 0))

## Dataset statistics

In [ ]:
unnormalized_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

unnormalized_dataset = CustomDataset(dataset, transform=unnormalized_transforms)
loader = DataLoader(unnormalized_dataset, batch_size=batch_size, shuffle=False)

mean_sum = torch.zeros(3)
std_sum = torch.zeros(3)
n_samples = 0

for imgs, _ in loader:
    imgs = imgs/255.0 if imgs.max() > 1 else imgs

    batch_samples = imgs.size(0)
    imgs = imgs.view(batch_samples, imgs.size(1), -1)
    
    mean_sum += imgs.mean(dim=[0, 2]) * batch_samples
    std_sum += imgs.std(dim=[0, 2]) * batch_samples
    n_samples += batch_samples

mean = mean_sum / n_samples
std = std_sum / n_samples

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),             
    transforms.RandomRotation(180),            
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

test_valid_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])


train_dataset = CustomDataset(train_dataset, transform=train_transform)
val_dataset = CustomDataset(val_dataset, transform=test_valid_transforms)
test_dataset = CustomDataset(test_dataset, transform=test_valid_transforms)

# Acceder e imprimir las primeras 5 muestras del dataset
for i in range(5):
    image, label = train_dataset[i]
    print(f"Muestra {i}: Imagen - {type(image)}, Dimensiones - {image.size()}, Etiqueta - {label}")

# Imprimir los valores de la media y la desviación estándar
print(f"Media: {mean}, Desviación Estándar: {std}")

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

# Verificar DataLoader
for images, labels in train_loader:
    print(f"Batch de imágenes: {images.shape}, Batch de etiquetas: {labels.shape}")
    break

# Carga una imagen de prueba
img, label = test_dataset[45]
show_img(img, label)

## Comprobaciones previas de la GPU

In [ ]:
print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo configurado para usar: {device}")

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

## Entrenamos

In [ ]:
early_stopping = EarlyStopping(patience=10, min_delta=.01)

In [ ]:
model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)

def callback(val_loss, val_acc):
    print(f"val_loss: {val_loss}, val_acc: {val_acc}")

def early_callback(): print("> Early stopping <")

train_loop(model, train_loader,
           optimizer=optimizer,
           criterion=criterion,
           num_epoch=num_epoch,
           device=device,
           early_stopping=early_stopping,
           early_callback=early_callback,
           callback=callback
           )